In [28]:
import sys
import functools
from collections import OrderedDict 

## 1. Декоратор @cached 

In [29]:
def cached(*args, **kwargs):
    max_count = 10
    if ('max_count' in kwargs):
        max_count = kwargs['max_count']
    elif len(args) > 0 and isinstance(args[0], int):
        max_count = args[0]
            
    class LruCache(object):
        
        def __init__(self, func):
            self.__func = func
            self.__cached = []
            functools.update_wrapper(self, func)
            
        def __call__(self, *args, **kwargs):
            res = self.__func(*args, **kwargs)
            if len(self.__cached) == max_count:
                self.__cached = self.__cached[1:]
            self.__cached.append(res)
            return res
        
        def cache(self):
            return self.__cached
        
    return LruCache
    

In [30]:
@cached(3)
def f(x):
    return x

@cached()
def g(x):
    return x

In [31]:
for i in range(10):
    f(i)
    g(i)
    
print(f.cache())
print(g.cache())

[7, 8, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [32]:
f.__name__

'f'

In [10]:
from functools import lru_cache

## 2. Декоратор @checked

In [44]:
def checked(*types):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            args_types = [type(arg) for arg in args]
            args_types += [type(arg) for arg in kwargs.values()]
            if len(types) != len(args_types):
                raise TypeError
            for t1, t2 in zip(types, args_types):
                if t1 != t2:
                    raise TypeError
            return func(*args, **kwargs)
        return wrapper
    return decorator

In [45]:
from typing import List

@checked(str, int, list)
def strange_func(a: str, b: int, c: List):
    pass

strange_func('asdf', 10, [])

In [49]:
@checked(str, int, list)
def strange_func(a: str, b: int, c: List):
    pass

try:
    strange_func('asdf', 10, 'asdf')
except TypeError:
    print('типы должны совпадать')

типы должны совпадать


In [48]:
@checked(str, int, list)
def strange_func(a: str, b: int, c: List, d: str):
    pass

try:
    strange_func('asdf', 10, [], 'asdf')
except TypeError:
    print('число аргументов должно совпадать')

число аргументов должно совпадать


## 3. Декоратор @Logger

In [84]:
class Logger(object):
    
    def __init__(self, dest=sys.stdout):
        self.__dest = dest
        
    def __call__(self, func):
        
        @functools.wraps(func)
        def callable(*args, **kwargs):
            res = func(*args, **kwargs)
            print("[log]\nfunction name: {}\nargs: {}\nkwargs: {}\nresult: {}".format(func.__name__, args, kwargs, res), file=self.__dest)
            return res
        
        return callable
        

In [85]:
@Logger()
def f(x):
    return x

In [86]:
f(5)

[log]
function name: f
args: (5,)
kwargs: {}
result: 5


5